In [1]:
import tensorflow as tf
import numpy as np

In [2]:
beta_true = np.array([2, 1, 3], dtype=np.float)
X = 0.2*np.random.randn(10, 3) + np.array([3, 1, 2])
Y = X.dot(beta_true) + 0.1*np.random.randn(10)

In [3]:
X

array([[ 2.84191287,  0.90577322,  2.00011667],
       [ 3.25694602,  1.20469387,  2.35623964],
       [ 3.20523898,  1.1305971 ,  2.28246314],
       [ 3.02773708,  1.35206348,  2.20828464],
       [ 3.21546814,  1.33037426,  1.97505291],
       [ 2.8921624 ,  1.17117094,  1.99926117],
       [ 3.0519642 ,  0.84734446,  2.14804009],
       [ 2.8785236 ,  1.26959117,  1.92257212],
       [ 3.4686792 ,  1.23811325,  1.96051698],
       [ 2.83270008,  0.88849026,  1.70203315]])

In [4]:
Y

array([ 12.63119895,  14.69139723,  14.44576831,  14.14313805,
        13.55357155,  12.81225004,  13.32734516,  12.74377427,
        14.17100097,  11.67448513])

In [5]:
beta_ols = np.linalg.inv(X.T.dot(X)).dot(X.T.dot(Y))
beta_ols

array([ 2.05996628,  0.94565561,  2.93361638])

$$
loss=\frac{1}{2N}(Y - X\beta)^T(Y-X\beta) \\
\frac{\partial loss}{\partial \beta}=\frac{1}{N}(X^TX\beta-X^TY)
$$

In [6]:
def grad_np(Y, X, beta):
    N = len(Y)
    return (X.T.dot(X).dot(beta) - X.T.dot(Y))/N

In [7]:
# estimate beta with gradient descent
beta_np = np.zeros(3, dtype=np.float)
for _ in range(1000):
    beta_np -= 0.1*grad_np(Y, X, beta_np)
beta_np # very close to true beta [2, 1, 3]

array([ 2.15488409,  0.91723683,  2.80772781])

## Tensorflow implementation

I want to show you how `tf.gradients` works in `Tensorflow`.

In [8]:
# OLS gradient descent with Tensorflow
tf.reset_default_graph()
tf_X = tf.constant(X, dtype=tf.float64, name="tf_X")
tf_Y = tf.constant(Y, dtype=tf.float64, name="tf_Y")
tf_beta = tf.Variable(tf.zeros(3, dtype=tf.float64), name="beta")
tf_N = tf.cast(tf.shape(tf_Y), tf.float64)[0]
tf_Y_hat = tf.reduce_sum(tf_X*tf_beta, 1)
tf_loss = tf.reduce_sum(tf.square(tf_Y - tf_Y_hat))/(2*tf_N)
tf_gradients = tf.gradients(tf_loss, [tf_beta])[0]

In [9]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [10]:
grad, _ = sess.run([tf_gradients, tf_beta])

In [11]:
np.allclose(grad_np(Y, X, np.zeros(3)), grad) # the same as numpy implementaion

True

In [12]:
# run iteration
for _ in range(1000):
    grad, beta = sess.run([tf_gradients, tf_beta])
    beta -= 0.1*grad
    _ = sess.run(tf_beta.assign(beta)) # this line is extremely slow....
beta

array([ 2.15488409,  0.91723683,  2.80772781])